In [27]:
import requests

In [28]:
#standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
standings_url = 'https://fbref.com/en/comps/31/Liga-MX-Stats'

In [29]:
data = requests.get(standings_url)

In [30]:
from bs4 import BeautifulSoup

In [31]:
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [33]:
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/dd5ca9bd/Monterrey-Stats',
 'https://fbref.com/en/squads/18d3c3a3/America-Stats',
 'https://fbref.com/en/squads/d9e1bd51/UANL-Stats',
 'https://fbref.com/en/squads/1be8d2e3/Pachuca-Stats',
 'https://fbref.com/en/squads/44b88a4e/Toluca-Stats',
 'https://fbref.com/en/squads/03b65ba9/Santos-Laguna-Stats',
 'https://fbref.com/en/squads/c02b0f7a/Guadalajara-Stats',
 'https://fbref.com/en/squads/fd7dad55/Leon-Stats',
 'https://fbref.com/en/squads/632f1838/Cruz-Azul-Stats',
 'https://fbref.com/en/squads/29bff345/FC-Juarez-Stats',
 'https://fbref.com/en/squads/73fd2313/Puebla-Stats',
 'https://fbref.com/en/squads/752db496/Necaxa-Stats',
 'https://fbref.com/en/squads/5d274ee4/Atletico-Stats',
 'https://fbref.com/en/squads/a42ddf2f/Tijuana-Stats',
 'https://fbref.com/en/squads/c9d59c6c/UNAM-Stats',
 'https://fbref.com/en/squads/7c76bc53/Atlas-Stats',
 'https://fbref.com/en/squads/f0297c23/Mazatlan-Stats',
 'https://fbref.com/en/squads/c3352ce7/Queretaro-Stats']

In [34]:
data = requests.get(team_urls[0])

In [35]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [36]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

C:\Users\Marco .LAPTOP-8I11FSGG\AppData\Local\Temp\ipykernel_29384\2495039547.py:1: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 1 of the file D:\Archivos\repos\my-data-science\Football-ML-v2.0\notebooks\scraping.ipynb. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(data.text)


In [37]:
data = requests.get(f"https://fbref.com{links[0]}")

In [38]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [39]:
shooting.head()

For Monterrey                                                           \
           Date   Time     Comp                         Round  Day Venue   
0    2022-07-03  19:05  Liga MX  Apertura 2022 Regular Season  Sun  Away   
1    2022-07-09  21:05  Liga MX  Apertura 2022 Regular Season  Sat  Home   
2    2022-07-17  17:00  Liga MX  Apertura 2022 Regular Season  Sun  Away   
3    2022-07-21  21:00  Liga MX  Apertura 2022 Regular Season  Thu  Away   
4    2022-07-26  19:00  Liga MX  Apertura 2022 Regular Season  Tue  Home   

                           ... Standard             Expected               \
  Result GF GA   Opponent  ...     Dist FK PK PKatt       xG npxG npxG/Sh   
0      L  3  4     Santos  ...     19.0  1  1     1      2.3  1.5    0.09   
1      W  3  2    América  ...     20.7  0  0     0      1.2  1.2    0.07   
2      W  1  0   Atlético  ...     24.1  0  1     1      1.7  1.0    0.06   
3      W  3  0  Querétaro  ...     16.8  1  0     0      1.7  1.7    0.12   
4      W  1  0     Puebla  ...     17.7  0  0     0      3.2  3.2    0.15   

               Unnamed: 25_level_0  
  G-xG np:G-xG        Match Report  
0  0.7     0.5        Match Report  
1  1.8     1.8        Match Report  
2 -0.7    -1.0        Match Report  
3  1.3     1.3        Match Report  
4 -2.2    -2.2        Match Report  

[5 rows x 26 columns]

In [14]:
shooting.columns = shooting.columns.droplevel()

In [15]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [16]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-07-03,19:05,Liga MX,Apertura 2022 Regular Season,Sun,Away,L,3.0,4.0,Santos,...,4-2-3-1,Jorge Pérez,Match Report,NaN,16,7,19.0,1,1,1
1,2022-07-09,21:05,Liga MX,Apertura 2022 Regular Season,Sat,Home,W,3.0,2.0,América,...,4-2-3-1,Marco Ortíz,Match Report,NaN,19,8,20.7,0,0,0
2,2022-07-17,17:00,Liga MX,Apertura 2022 Regular Season,Sun,Away,W,1.0,0.0,Atlético,...,4-2-3-1,Fernando Guerrero,Match Report,NaN,18,4,24.1,0,1,1
3,2022-07-21,21:00,Liga MX,Apertura 2022 Regular Season,Thu,Away,W,3.0,0.0,Querétaro,...,4-2-3-1,Luis Santander,Match Report,NaN,15,6,16.8,1,0,0
4,2022-07-26,19:00,Liga MX,Apertura 2022 Regular Season,Tue,Home,W,1.0,0.0,Puebla,...,4-4-2,Óscar Macias,Match Report,NaN,22,7,17.7,0,0,0


In [17]:
years = list(range(2022, 2020, -1))
all_matches = []

In [56]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [18]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

C:\Users\Marco .LAPTOP-8I11FSGG\AppData\Local\Temp\ipykernel_29384\1338002569.py:4: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 4 of the file D:\Archivos\repos\my-data-science\Football-ML-v2.0\notebooks\scraping.ipynb. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(data.text)
C:\Users\Marco .LAPTOP-8I11FSGG\AppData\Local\Temp\ipykernel_29384\1338002569.py:18: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use

In [24]:
len(all_matches)

36

In [21]:
match_df = pd.concat(all_matches)

In [22]:
match_df.columns = [c.lower() for c in match_df.columns]

In [23]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team


In [25]:
match_df.to_csv("matches.csv")

In [26]:
all_matches

[Empty DataFrame
 Columns: [Date, Time, Comp, Round, Day, Venue, Result, GF, GA, Opponent, xG, xGA, Poss, Attendance, Captain, Formation, Referee, Match Report, Notes, Sh, SoT, Dist, FK, PK, PKatt, Season, Team]
 Index: []
 
 [0 rows x 27 columns],
 Empty DataFrame
 Columns: [Date, Time, Comp, Round, Day, Venue, Result, GF, GA, Opponent, xG, xGA, Poss, Attendance, Captain, Formation, Referee, Match Report, Notes, Sh, SoT, Dist, FK, PK, PKatt, Season, Team]
 Index: []
 
 [0 rows x 27 columns],
 Empty DataFrame
 Columns: [Date, Time, Comp, Round, Day, Venue, Result, GF, GA, Opponent, xG, xGA, Poss, Attendance, Captain, Formation, Referee, Match Report, Notes, Sh, SoT, Dist, FK, PK, PKatt, Season, Team]
 Index: []
 
 [0 rows x 27 columns],
 Empty DataFrame
 Columns: [Date, Time, Comp, Round, Day, Venue, Result, GF, GA, Opponent, xG, xGA, Poss, Attendance, Captain, Formation, Referee, Match Report, Notes, Sh, SoT, Dist, FK, PK, PKatt, Season, Team]
 Index: []
 
 [0 rows x 27 columns],
 Emp